In [1]:
import torch
torch.__version__
CUDA_VISIBLE_DEVICES=2

In [2]:
#from google.colab import drive
#drive.mount("/content/drive")
#import os
#os.chdir(os.path.join("/", "content", "drive", "MyDrive", "Colab Notebooks"))

In [3]:
!pip install -U spacy[cuda102]
!python -m spacy download en_core_web_sm
!python -m spacy download zh_core_web_sm

Requirement already up-to-date: spacy[cuda102] in /home/deponce/anaconda3/lib/python3.8/site-packages (3.1.1)
2021-07-28 20:23:12.294298: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
2021-07-28 20:23:16.373858: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_sm')


In [4]:
from zh_dataset import ZhEnDataset, BaggingDataset


Language Translation with nn.Transformer and torchtext
======================================================

This tutorial shows, how to train a translation model from scratch using
Transformer. We will be using `Multi30k <http://www.statmt.org/wmt16/multimodal-task.html#task1>`__ 
dataset to train a German to English translation model.



Data Sourcing and Processing
----------------------------

`torchtext library <https://pytorch.org/text/stable/>`__ has utilities for creating datasets that can be easily
iterated through for the purposes of creating a language translation
model. In this example, we show how to use torchtext's inbuilt datasets, 
tokenize a raw text sentence, build vocabulary, and numericalize tokens into tensor. We will use
`Multi30k dataset from torchtext library <https://pytorch.org/text/stable/datasets.html#multi30k>`__
that yields a pair of source-target raw sentences. 





In [5]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import Multi30k
from typing import Iterable, List


SRC_LANGUAGE = 'en'
TGT_LANGUAGE = 'zh'

# Place-holders
token_transform = {}
vocab_transform = {}


# Create source and target language tokenizer. Make sure to install the dependencies.
# pip install -U spacy
# python -m spacy download en_core_web_sm
# python -m spacy download de_core_news_sm
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='zh_core_web_sm')


# helper function to yield list of tokens
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    for data_sample in data_iter:
        yield token_transform[language](data_sample[language_index[language]])

# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']
 
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # Training data Iterator 
    train_iter = ZhEnDataset(split='train', start=0, end=150000)
    # Create torchtext's Vocab object 
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                    # min_freq=1,
                                                    min_freq=3,
                                                    specials=special_symbols,
                                                    special_first=True)
# Set UNK_IDX as the default index. This index is returned when the token is not found. 
# If not set, it throws RuntimeError when the queried token is not found in the Vocabulary. 
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  vocab_transform[ln].set_default_index(UNK_IDX)

Seq2Seq Network using Transformer
---------------------------------

Transformer is a Seq2Seq model introduced in `“Attention is all you
need” <https://papers.nips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf>`__
paper for solving machine translation tasks. 
Below, we will create a Seq2Seq network that uses Transformer. The network
consists of three parts. First part is the embedding layer. This layer converts tensor of input indices
into corresponding tensor of input embeddings. These embedding are further augmented with positional
encodings to provide position information of input tokens to the model. The second part is the 
actual `Transformer <https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html>`__ model. 
Finally, the output of Transformer model is passed through linear layer
that give un-normalized probabilities for each token in the target language. 




In [6]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

class PositionEmbedding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 maxlen: int = 5000):
        super(PositionEmbedding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = nn.Parameter(torch.Tensor(maxlen, emb_size))
        pos_embedding = pos_embedding.unsqueeze(-2)
        
        self.rfegister_buffer('pos_embedding', pos_embedding)
        torch.nn.init.xavier_normal_(pos_embedding)

    def forward(self, token_embedding: Tensor):
        return token_embedding + self.pos_embedding[:token_embedding.size(0), :]

In [7]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network 
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionEmbedding(
            emb_size)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None, 
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

During training, we need a subsequent word mask that will prevent model to look into
the future words when making predictions. We will also need masks to hide
source and target padding tokens. Below, let's define a function that will take care of both. 




In [8]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

Let's now define the parameters of our model and instantiate the same. Below, we also 
define our loss function which is the cross-entropy loss and the optmizer used for training.




In [9]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])

EMB_SIZE = 512
# EMB_SIZE = 128
# EMB_SIZE = 64

NHEAD = 8

FFN_HID_DIM = 512
# FFN_HID_DIM = 128
# FFN_HID_DIM = 64

BATCH_SIZE = 32
# BATCH_SIZE = 40

NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, 
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [10]:
import numpy as np
model_parameters = filter(lambda p: p.requires_grad, transformer.parameters())
params = sum([np.prod(transformer.size()) for transformer in model_parameters])
params

85027047

In [11]:
SRC_VOCAB_SIZE, TGT_VOCAB_SIZE

(42548, 49383)

Collation
---------

As seen in the ``Data Sourcing and Processing`` section, our data iterator yields a pair of raw strings. 
We need to convert these string pairs into the batched tensors that can be processed by our ``Seq2Seq`` network 
defined previously. Below we define our collate function that convert batch of raw strings into batch tensors that
can be fed directly into our model.   




In [12]:
from torch.nn.utils.rnn import pad_sequence

# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]), 
                      torch.tensor(token_ids), 
                      torch.tensor([EOS_IDX])))

# src and tgt language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor


# function to collate data samples into batch tesors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

Let's define training and evaluation loop that will be called for each 
epoch.




In [13]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer, train_iter):
    model.train()
    losses = 0
    
    # train_iter = ZhEnDataset(split='train', start=15000, end=65000)
    train_iter.resample()

    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    
    for src, tgt in train_dataloader:
        # print(src.shape)
        # print(tgt.shape)
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(train_dataloader)


def evaluate(model):
    model.eval()
    losses = 0

    val_iter = ZhEnDataset(split='valid', start=15000, end=25000)
    val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
        
        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(val_dataloader)

Now we have all the ingredients to train our model. Let's do it!




In [15]:
from timeit import default_timer as timer
NUM_EPOCHS = 60
train_iter = BaggingDataset(size=25000)

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer, train_iter)
    end_time = timer()
    val_loss = evaluate(transformer)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))


# function to generate output sequence using greedy algorithm 
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

Epoch: 1, Train loss: 5.618, Val loss: 5.492, Epoch time = 264.943s
Epoch: 2, Train loss: 5.502, Val loss: 5.379, Epoch time = 264.303s
Epoch: 3, Train loss: 5.411, Val loss: 5.296, Epoch time = 263.047s
Epoch: 4, Train loss: 5.316, Val loss: 5.210, Epoch time = 264.250s
Epoch: 5, Train loss: 5.245, Val loss: 5.135, Epoch time = 265.275s
Epoch: 6, Train loss: 5.188, Val loss: 5.083, Epoch time = 265.897s
Epoch: 7, Train loss: 5.142, Val loss: 5.034, Epoch time = 265.940s
Epoch: 8, Train loss: 5.087, Val loss: 4.978, Epoch time = 267.960s
Epoch: 9, Train loss: 5.050, Val loss: 4.936, Epoch time = 265.143s
Epoch: 10, Train loss: 5.019, Val loss: 4.899, Epoch time = 267.112s
Epoch: 11, Train loss: 4.965, Val loss: 4.850, Epoch time = 265.850s
Epoch: 12, Train loss: 4.927, Val loss: 4.814, Epoch time = 265.323s
Epoch: 13, Train loss: 4.886, Val loss: 4.787, Epoch time = 258.148s
Epoch: 14, Train loss: 4.863, Val loss: 4.763, Epoch time = 259.955s
Epoch: 15, Train loss: 4.830, Val loss: 4.7

In [16]:
for epoch in range(1, 11):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer, train_iter)
    end_time = timer()
    val_loss = evaluate(transformer)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))

Epoch: 1, Train loss: 4.321, Val loss: 4.223, Epoch time = 230.446s
Epoch: 2, Train loss: 4.311, Val loss: 4.210, Epoch time = 230.586s
Epoch: 3, Train loss: 4.310, Val loss: 4.206, Epoch time = 229.093s
Epoch: 4, Train loss: 4.298, Val loss: 4.211, Epoch time = 230.587s
Epoch: 5, Train loss: 4.308, Val loss: 4.193, Epoch time = 230.136s
Epoch: 6, Train loss: 4.292, Val loss: 4.186, Epoch time = 229.954s
Epoch: 7, Train loss: 4.282, Val loss: 4.179, Epoch time = 228.183s
Epoch: 8, Train loss: 4.280, Val loss: 4.168, Epoch time = 230.863s
Epoch: 9, Train loss: 4.269, Val loss: 4.164, Epoch time = 229.469s
Epoch: 10, Train loss: 4.251, Val loss: 4.160, Epoch time = 229.337s


In [20]:
for epoch in range(1, 50):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer, train_iter)
    end_time = timer()
    val_loss = evaluate(transformer)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))

Epoch: 1, Train loss: 4.261, Val loss: 4.167, Epoch time = 247.087s
Epoch: 2, Train loss: 4.252, Val loss: 4.148, Epoch time = 244.961s
Epoch: 3, Train loss: 4.251, Val loss: 4.151, Epoch time = 245.715s
Epoch: 4, Train loss: 4.252, Val loss: 4.139, Epoch time = 244.917s
Epoch: 5, Train loss: 4.245, Val loss: 4.138, Epoch time = 245.211s
Epoch: 6, Train loss: 4.229, Val loss: 4.141, Epoch time = 244.054s
Epoch: 7, Train loss: 4.238, Val loss: 4.128, Epoch time = 245.910s
Epoch: 8, Train loss: 4.222, Val loss: 4.123, Epoch time = 246.503s
Epoch: 9, Train loss: 4.219, Val loss: 4.124, Epoch time = 244.251s
Epoch: 10, Train loss: 4.211, Val loss: 4.124, Epoch time = 243.484s
Epoch: 11, Train loss: 4.212, Val loss: 4.119, Epoch time = 245.696s
Epoch: 12, Train loss: 4.210, Val loss: 4.112, Epoch time = 247.243s
Epoch: 13, Train loss: 4.207, Val loss: 4.113, Epoch time = 245.407s
Epoch: 14, Train loss: 4.200, Val loss: 4.106, Epoch time = 246.102s
Epoch: 15, Train loss: 4.192, Val loss: 4.0

In [22]:
print(translate(transformer, "Attention is all you need."))
print(translate(transformer, "How are you doing?"))
print(translate(transformer, "Translate this from English to Chinese"))
print(translate(transformer, "Please copy this code, switch to your application and paste it there."))
print(translate(transformer, "To continue signing in to your Crowdmark account, please select your school."))
print(translate(transformer, "Help people interested in this repository understand your project by adding a README."))
print(translate(transformer, "Get notified in real-time when visitors press your doorbell or trigger the built-in motion sensors."))

 注意 是 所有 需要 的 。 
 你 怎么 做 ？ 
 从 英语 翻译 到 英语 
 请 将 这 份 代码 复制 到 您 的 应用 程序 中 ， 并 将 它 复制 到 您 的
 请 继续 选择 你 的 学校 账户 ， 请 继续 你 的 帐户 。 
 帮助 您 理解 项目 中 的 人们 对 <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> 感兴趣 。 
 当 你 的 实时 移动 传感器 或 <unk> <unk> <unk> <unk> <unk> 时 ， 请 通知 游客 。 


References
----------

1. Attention is all you need paper.
   https://papers.nips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf
2. The annotated transformer. https://nlp.seas.harvard.edu/2018/04/03/attention.html#positional-encoding



In [23]:
PATH = "./models/bagging-150k-512-8-512ebmedding.pt"
torch.save(transformer.state_dict(), PATH)

In [24]:
PATH = "./models/bagging-150k-512-8-512ebmedding.pt"
transformer.load_state_dict(torch.load(PATH))

<All keys matched successfully>